# Recomendación de plan

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
* [Etapa 2. Segmentación de los datos](#data_segmentation)
* [Etapa 3. Elección del mejor modelo](#data_model)
    * [3.1. Árbol de decisión](#data_model_tree)
    * [3.2. Bosque aleatorio](#data_model_forest)
    * [3.3. Regresicón logística](#data_model_regression)
    * [3.4. Calidad del modelo](#data_model_test)
* [Etapa 4. Prueba de cordura](#data_sanity)
* [Etapa 5. Conclusiones](#data_conclusion)

## Introducción <a id='intro'></a>

La compañía móvil Megaline tiene el objetivo de desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar alguno de los dos planes con los que cuenta: Smart o Ultra.

**Objetivo**

El objetivo de este proyecto es crear un modelo con la mayor exactitud posible que elija el plan correcto, para esto se cuenta con los datos de del comportamiento de los suscriptores que ya se han cambiado a los planes nuevos.

**Etapas**
Los datos se almacenan en el archivo `/datasets/users_behavior.csv`. Ya se han procesado los datos con anterioridad por lo que se procederá a crear el modelo.
 
El proyecto consistirá en cinco etapas:
 1. Descripción de los datos.
 2. Segementación de los datos.
 3. Elección del mejor modelo.
 4. Prueba de cordura del modelo.
 5. Conclusión general.

[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Se lee el archivo y se guardará en la variable `df`.

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

Se imprimirá la información general de `df` y sus primeras 10 filas.

In [3]:
df.info()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0


El dataset cuenta con 3,214 filas, cada una de las cuales contiene información del comportamiento mensual sobre un usuario. Las 5 columnas son:

1. `'calls'` - cantidad de llamadas realizadas.
2. `'minutes'` - total de minutos utilizados.
3. `'messages'` - cantidad de mensajes enviados.
4. `'mb_used'` - trafico de internet utilizado en MB.
5. `'is_ultra'` - plan para el mes actual (Ultra - 1, Smart - 0).

Como se había mencionado, los datos ya han sido preprocesados, por ello ninguna de las columnas contiene valores asuentes, además, los tipos de datos son correctos.

[Volver a Contenidos](#back)

## Etapa 2. Segmentación de los datos <a id='data_segmentation'></a>

En esta etapa se segmentará el dataset `df` en tres conjuntos, siguiendo una proporción 3:1:1. Es decir, se tendrán lo siguiente:

1. un conjunto de entrenamiento que representa el 60% de los datos,
2. un conjunto validación que representa el 20% y
3. un conjunto de prueba que representa el 20% restante.

In [4]:
# Primero se segmenta df para obtener el conjunto de entrenamiento (60%)
df_train, df2 = train_test_split(df, test_size=0.40, random_state=12345)
# Ahora se segmenta el resto (df2) a la mitad para obtener los conjuntos de validación y de prueba, cada uno con el 20% de df
df_valid, df_test = train_test_split(df2, test_size=0.50, random_state=12345)

El conjunto de entrenamiento es `df_train`, el de validación es `df_valid` y el de prueba es `df_test`.

[Volver a Contenidos](#back)

## Etapa 3. Elección del mejor modelo <a id='data_model'></a>

En esta etapa se elegirá el mejor modelo, para ello se ajustarán hiperparámetros y se calculará la exactitud del modelo. Se filtrarán los datasets de entrenamiento y validación para establecer las características y objetivos del modelo.

In [5]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

### 3. 1. Árbol de decisión <a id='data_model_tree'></a>

En esta sección se creará un modelo con un árbol de decisión y se ajustará la profundidad máxima para obtener la mejor exactitud.

In [6]:
for i in range(1, 7):
    model = DecisionTreeClassifier(random_state=54321, max_depth=i)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    print(f'max_depth = {i} :', result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255


El umbral de exactitud para el modelo buscado es del 75% y se observa que con profundidad máxima igual a 3 se alzcanza un 78.5% de exactitud. Se analizará si con un bosque aleatorio se obtiene una exactitud mayor y además evitar sobrejuste. 

### 3. 2. Bosque aleatorio <a id='data_model_forest'></a>

Se ajustará el número de árboles para el bosque (hiperparámetro) y se evaluará la exactitud.

In [8]:
best_score = 0
best_est = 0
best_depth = 0
for i in range(1, 11): #rango de profundidad
    for est in range(10, 101, 10): # rango del hiperparámetro, comenzando en 10, terminando en 100 y dando saltos de 10 en 10
        model = RandomForestClassifier(random_state=54321, max_depth=i, n_estimators=est) # n_estimators ajusta el número de árboles
        model.fit(features_train, target_train)
        score = model.score(features_valid, target_valid) # exactitud en el conjunto de validación
        if score > best_score:
            best_score = score
            best_est = est
            best_depth = i

print("Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}, max_depth = {}): {}".format(best_est, best_depth, best_score))

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 40, max_depth = 7): 0.8055987558320373


En este caso, con 40 árboles y una profundidad máxima igual a 7 se alcanza un 80% de exactitud, mayor que con un sólo árbol de decisión y sin sobreajuste. Sin embargo, también se considerará la regresión logistíca.

### 3. 3. Regresión logística <a id='data_model_regression'></a>

Se entrenará al modelo mediante la regresión logística y se calculará su exactitud en el conjunto de validación.

In [9]:
model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train)
score_valid = model.score(features_valid, target_valid) #exactitud en el conjunto de validación

print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)

Exactitud del modelo de regresión logística en el conjunto de validación: 0.7573872472783826


Aunque este modelo tiene mayor velocidad de ejecución, su exactitud (75%) es más baja que con el bosque aleatorio.

### 3. 4. Calidad del modelo <a id='data_model_test'></a>

El modelo con mayor exactitud es el que utiliza el algoritmo de bosque aleatorio con 40 áborles. Por lo que se decide que este es el mejor modelo.

In [11]:
final_model = RandomForestClassifier(random_state=54321, max_depth=7, n_estimators=40)
final_model.fit(features_train, target_train)

RandomForestClassifier(max_depth=7, n_estimators=40, random_state=54321)

Se comprobará la calidad del modelo utilzando el conjunto de prueba.

In [12]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

test_predictions = final_model.predict(features_test)
score_test = final_model.score(features_test, target_test)

print("Exactitud del modelo final en el conjunto de prueba: {}".format(score_test))

Exactitud del modelo final en el conjunto de prueba: 0.8055987558320373


La exactitud del modelo final en el conjunto de prueba es del 80%, igua a la exactitud en el conjunto de validación. Por esta razón se concluye que este es el mejor modelo.

[Volver a Contenidos](#back)

## Etapa 4. Prueba de cordura <a id='data_sanity'></a>

Se realizará una prueba de cordura para comprobar que el modelo funciona mejor que la casualidad. Para ello, se utilizarán los datos del conjunto de prueba. Se obtendrán la cantidad de clientes con el plan `'ultra'` y aquellos con el plan `'smart'`.

In [13]:
ultra_target = (target_test == 1)
smart_target = (target_test == 0)

print('Clientes con plan ultra:', ultra_target.sum())
print('Clientes con plan smart:', smart_target.sum())

Clientes con plan ultra: 203
Clientes con plan smart: 440


La cantidad de clientes con el plan `'smart'` es mayor que la de los clientes con el plan `'ultra'`. Se supondrá que se tiene un modelo aleatorio cuyas predicciones son sólo 0 (plan `'smart'`). Con esta información se calculará la exactitud de este modelo aleatorio (casualidad).

In [14]:
smart_score = smart_target.sum() / len(target_test)
print(f'Exactitud del modelo aleatorio: {smart_score}')

Exactitud del modelo aleatorio: 0.6842923794712286


La casualidad tiene una exactitud del 68%, por lo tanto, el modelo elegido `final_model` con exactitud del 80% pasa la prueba de cordura.

## Etapa 5. Conclusión <a id='data_conclusion'></a>

El conjunto de datos se dividió en tres partes: conjunto de entrenamiento (60%), conjunto de validación (20%) y conjunto de prueba (20%). Con los datos del conjunto de entrenamiento se crearon y entrenaron los modelos, usando los tres tipos de algoritmos de clasificación: árbol de decisión, bosque aleatorio y regresión lógistica.

Para elegir el mejor modelo se ajustaron hiperparámetros y se comprobó la exactitud de cada uno de los modelos con los datos del conjunto de validación. En el árbol de decisión se concluyó que la mayor exactitud se obtuvó con profundidad máxima igual a 3, mientras que para el bosque aleatorio, se alcanzó la mayor exactitud con 40 árboles y profundidad máxima igual a 7.

Comprando la exactitud de los tres algoritmos, se concluyó que el mejor modelo (mayor exactitud) es el bosque aleatorio (40 árboles y produndidad máxima 7).

Se comprobó la calidad del modelo con los datos del conjunto de prueba, obteniendo una exactitud del 80%, igual a la exactitud obtenida con el conjunto de validación.

Finalmente, se realizó una prueba de cordura para verificar que el modelo funciona mejor que la casualidad. La exactitud de la casualidad fue de 68%, más baja que la del modelo. Por lo tanto, el modelo elegido es el mejor para predecir el plan de cada usuario.